In [1]:
from pyscf import gto
from qiskit import QuantumCircuit
from qc_practice.simulator import simulator
from qc_practice.measure.hamiltonian import hamiltonian, pyscf_luncher
from qc_practice.measure.measure import measure
from qc_practice.simulator.simulator import QASM
from qc_practice.ansatz import UCCSD
from qc_practice.profile import Profile

# Define the molecule
mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')

info = pyscf_luncher(mol)
h_pauli = hamiltonian(info)
ansatz = UCCSD()
simulator = QASM()
profile = Profile()
profile.num_elec = info['num_elec']
profile.num_orb = info['num_orb']

# Define the quantum circuit
qc = QuantumCircuit(profile.num_orb*2, profile.num_orb*2)
for i in range(profile.num_elec//2):
    qc.x(i)
    qc.x(i + profile.num_elec)

def calculation(coeff):
    qc = QuantumCircuit(profile.num_orb*2, profile.num_orb*2)
    for i in range(profile.num_elec//2):
        qc.x(i)
        qc.x(i + profile.num_elec)

    ansatz.mapping(profile, coeff)
    ansatz.ansatz(qc, profile, coeff)

    energy = measure(qc, h_pauli, simulator, parallel=True)
    print(energy, end='\r', flush=True)
    return energy

for i in range(2):
    coeff = ansatz.generate_coeff(profile)
    optimized = ansatz.call_optimizer(calculation, coeff, 'powell')

    print('\nCompleted')
    print(f'Energy: {optimized.fun}\n')

-1.8918539658846627
Completed
Energy: -1.8941507995888263

-1.8928703639577544
Completed
Energy: -1.8938942097303384



In [2]:
from pyscf import gto
from qiskit import QuantumCircuit
from qc_practice.measure.hamiltonian import hamiltonian
from qc_practice.measure.measure import measure
from qc_practice.simulator import QASM
from qc_practice.ansatz import UCCSD
from qc_practice.profile import Profile

class Test:
    def __init__(self):
        mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
        self.profile = Profile(mol)
        self.h_pauli = hamiltonian(self.profile)
        self.ansatz = UCCSD()
        self.simulator = QASM()
        self.profile_list = []

    def calculation(self, coeff):
        qc = QuantumCircuit(profile.num_orb*2, profile.num_orb*2)
        for i in range(profile.num_elec//2):
            qc.x(i)
            qc.x(i + profile.num_elec)

        self.ansatz.mapping(profile, coeff)
        self.ansatz.ansatz(qc, profile, coeff)

        energy = measure(qc, self.h_pauli, self.simulator, parallel=True)
        print(energy, end='\r', flush=True)
        self.profile.energy_elec = energy
        self.profile.coeff = coeff
        self.profile.circuit = qc
        return energy

    def run(self, coeff):
        for i in range(2):
            coeff2 = coeff.copy()
            optimized = self.ansatz.call_optimizer(self.calculation, coeff2, 'powell')
            self.profile.energy_elec = optimized.fun
            self.profile.coeff = optimized.x
            self.profile_list.append(self.profile)
            for j in range(i):
                overlap = self.simulator.swap_test(self.profile_list[0], self.profile_list[1])
                print(overlap)

            print('\nCompleted')
            print(f'Energy: {optimized.fun}\n')

mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
profile = Profile(mol)

test = Test()
coeff = UCCSD().generate_coeff(profile)

test.run(coeff)

-1.8911868638494949
Completed
Energy: -1.8931880573304367

01.8888131748175159

Completed
Energy: -1.892821477774264



In [3]:
import time
from multiprocessing import Pool
from qiskit import QuantumCircuit
from qiskit_aer import AerProvider

backend = AerProvider().get_backend('qasm_simulator')

def single_measure(num):
    time.sleep(1)
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    # backend = AerProvider().get_backend('qasm_simulator')
    result = backend.run(qc, shots=100000).result().get_counts()
    print(num)
    print(f'{result}\n')

def parallel_measure():
    with Pool(2) as p:
        p.map(single_measure, range(2))

    single_measure(2)

parallel_measure()

/tmp/ipykernel_4850/792659838.py:6: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = AerProvider().get_backend('qasm_simulator')


In [1]:
from pyscf import gto
from qc_practice import VQE, VQD
from qc_practice.ansatz import UCCSD
from qc_practice.simulator import QASM, StateVector

mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
vqe = VQE(mol)
vqe.ansatz = UCCSD()
vqe.simulator = StateVector()
vqe.run()



Starting VQE Calculation

Ansatz: UCCSD
Simulator: StateVector
Optimizer: powell

Iteration: 80, Converged!!         
Total Energy: -1.136189454

Elapsed time: 0:00:07


Singlet_0

In [1]:
from pyscf import gto
from qc_practice import VQE, VQD
from qc_practice.ansatz import UCCSD, UpCCGSD
from qc_practice.simulator import QASM, StateVector

mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
vqd = VQD(mol)
vqd.ansatz = UCCSD()
vqd.simulator = StateVector()
vqd.nstates = 2
vqd.run()



Starting VQD Calculation

Ansatz: UCCSD
Simulator: StateVector
Optimizer: powell
nstates: 2

State 0:
Iteration: 80, Converged!!         
Total Energy: -1.136189454

State 1:
Iteration: 101, Converged!!         
Total Energy: -0.299452480

Final State Energies:
State 0: -1.136189454
State 1: -0.299452480

Elapsed time: 0:00:18


[Singlet_0, Singlet_1]

In [1]:
from pyscf import gto
from qc_practice.ssvqe_2 import SSVQE
from qc_practice.ansatz import UCCSD, UpCCGSD

mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
ssvqe = SSVQE(mol)
ssvqe.ansatz = UpCCGSD()

ssvqe.run()


Starting SSVQE Calculation

Ansatz: UCCSD
Simulator: QASM
Optimizer: powell
Koopmans: False
nstates: 4
Active Space: [1, 1]
weights: [9, 3, 2, 1]

Iteration: 1
State 0: -1.117044734
State 1: -0.299729931
State 2: -0.299738881
State 3:  0.564786935

Iteration: 2
State 0: -1.008355587
State 1: -0.191290504
State 2: -0.401796483
State 3:  0.441380001

Iteration: 3
State 0: -1.011537569
State 1: -0.180354581
State 2: -0.402221480
State 3:  0.450931358

Iteration: 4
State 0: -0.672652119
State 1:  0.169278618
State 2: -0.749168705
State 3:  0.098561152

Iteration: 5
State 0: -1.117995701
State 1: -0.299040778
State 2: -0.297626674
State 3:  0.565583488

Iteration: 6
State 0: -1.116310832
State 1: -0.299622530
State 2: -0.298342676
State 3:  0.563721882

Iteration: 7
State 0: -1.095309839
State 1: -0.273616926
State 2: -0.322256306
State 3:  0.540724989

Iteration: 8
State 0: -1.113852391
State 1: -0.297337903
State 2: -0.303183597
State 3:  0.559278063

Iteration: 9
State 0: -1.116787780
S